In [7]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
#imports section
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import cvxpy as cvx
import re, os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from math import *
from datetime import datetime, date, time, timedelta
from time import sleep
from mvo_utils_v2 import *
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [13]:
# Global variables
log = True
refresh_pricing = False
universe = 'spy-sectors'  # 'spy_sectors OR 'ark_etfs'
active_etf = None  # ETF name OR None for broad market

#variables for performance analysis
lookback = 60
net_exposure = 0
leverage = 1
min_weight = -0.05
max_weight = 0.05  # default optimization vars
min_len = 3
pos_filter = 0.01  # cleaning variables
rebal_frequency = 'W-' + datetime.now().strftime(
    '%a')  # this is used during the historical allocations
gamma_vals = np.logspace(0.5, 2, num=100)  # 1 to 100 range

In [14]:
# load spy sector components
# would be nice to load a JSON with configuration
ticker_map = {
    'benchmark': ['SPY'],
    'equity': ['VTI','VTV','VOE','VBR','VEA','VWO'],
    'fixed_income': ['VTIP', 'SHV', 'MUB', 'LQD', 'BNDX', 'EMB'],
    'spy-sectors': ['XLE', 'XLU', 'XLK', 'XLB', 'XLP', 'XLY', 'XLI', 'XLV', 'XLF', 'XLRE'],
    'ark-etfs': ['ARKG', 'ARKK', 'ARKQ', 'ARKW']
}

config = {
    'spy-sectors' : {
        'hold_cols': ['Symbol','Company Name', 'Index Weight'],
        'hold_format': r'holdings-spy-',
        'idx_col': 'Symbol',
        'fname': 'spy-sectors',
        'skiprows': 1
    },
    'ark-etfs' : {
        'hold_cols': ['ticker','company', 'weight(%)'],
        'hold_format': r'holdings-ark-',
        'idx_col': 'ticker',
        'fname': 'ark-etfs',
        'skiprows': 0
    }
}

hold_cols = config[universe]['hold_cols']
hold_format = config[universe]['hold_format']
idx_col = config[universe]['idx_col']
fname = config[universe]['fname']
skiprows = config[universe]['skiprows']

companies = load_components(
    companies, hold_format, hold_cols, 
    idx_col, ticker_map[universe], srows=skiprows)
print("Companies loaded:", companies.shape)
px_etfs = load_pricing(fname + '.csv', 'Date')
px_spy = load_pricing('SPY.csv', 'Date')

Companies loaded: (1010, 3)
Loaded pricing for spy-sectors.csv, with shape (2605, 10)
Loaded pricing for SPY.csv, with shape (69, 1)


In [15]:
# Load pricing for components of each sectors
px_etfs = load_pricing(universe + '.csv', 'Date')
px_spy = load_pricing('SPY.csv', 'Date')
consol_px = clean_nas(load_consol_px(ticker_map, universe))
if(active_etf != None): companies = companies[companies['ETF']==active_etf] # filter by selected ETF
tickers = companies.index.tolist()
tickers=[i for i in tickers if i not in ['BF.b','BRK.b']] ## esta es una correcion extra para no incluir 'BF.b','BRK.b'
consol_px = consol_px[list(set(tickers))]

Loaded pricing for spy-sectors.csv, with shape (2605, 10)
Loaded pricing for SPY.csv, with shape (69, 1)
Loaded pricing for XLE-hold-pricing.csv, with shape (2605, 31)
Loaded pricing for XLU-hold-pricing.csv, with shape (2605, 28)
Loaded pricing for XLK-hold-pricing.csv, with shape (2605, 73)
Loaded pricing for XLB-hold-pricing.csv, with shape (2605, 25)
Loaded pricing for XLP-hold-pricing.csv, with shape (2605, 34)
Loaded pricing for XLY-hold-pricing.csv, with shape (2605, 82)
Loaded pricing for XLI-hold-pricing.csv, with shape (2605, 70)
Loaded pricing for XLV-hold-pricing.csv, with shape (2605, 61)
Loaded pricing for XLF-hold-pricing.csv, with shape (2605, 68)
Loaded pricing for XLRE-hold-pricing.csv, with shape (2605, 33)


In [16]:
weights = np.zeros((len(consol_px.columns), 1))
np_weights = np.array(weights)
lb_rets = consol_px.sort_index().pct_change().dropna()
lb_weights = pd.DataFrame(np_weights.T, index=[lb_rets.index[-1]], columns=lb_rets.columns)

In [17]:
# illustration of how two allocation frames can be combined
df1 = pd.DataFrame(np.arange(0,6,1).reshape(2,3), index=[1,2], columns=list('abc'))
df2 = pd.DataFrame(np.arange(6,12,1).reshape(2,3), index=[1,2], columns=list('cde'))
df1.combine_first(df2)

,a,b,c,d,e
1,0,1,2,7.0,8.0
2,3,4,5,10.0,11.0


In [18]:
# illustration of how two allocation frames can be combined
df1 = pd.DataFrame(np.arange(0,6,1).reshape(2,3), index=[1,2], columns=list('abc'))
df2 = pd.DataFrame(np.arange(6,12,1).reshape(2,3), index=[3,4], columns=list('cde'))
df1.combine_first(df2)

,a,b,c,d,e
1,0.0,1.0,2.0,NaN,NaN
2,3.0,4.0,5.0,NaN,NaN
3,NaN,NaN,6.0,7.0,8.0
4,NaN,NaN,9.0,10.0,11.0


In [ ]:
# recreate performance for a given net exposure
ne = '0.8'
fname = FNAME_ALLOC.format(BACKTEST_PATH, ne, time_frame, rebal_frequency)
allocs = pd.read_csv(fname, index_col='Date', parse_dates=True)
fname = FNAME_RETURN.format(BACKTEST_PATH, ne, time_frame, rebal_frequency)
returns = pd.read_csv(fname, index_col='Date', parse_dates=True)
time_series = calc_port_performance(returns, allocs).to_frame()
ts, te = '2017-06', '2017-08'
time_series[ts:te].plot()
returns.loc[returns[ts:te].abs().max(axis=1).argmax()].argmax()

In [ ]:
# since there are two jumps, run this code twice
ts = net_exp_port[ne].pct_change().argmax()
date_idx = ts.strftime('%F')
i = len(net_exp_port[ne].loc[:date_idx])
index_val = net_exp_port[ne].iloc[i - 2]
adjusted = compound(net_exp_port[ne].loc[date_idx:]) * index_val
net_exp_port[ne].loc[adjusted.index] = adjusted.values

In [ ]:
time_frame = '2007-12-25'; rebal_frequency = 'BM'
fname = FNAME_INDEX.format(BACKTEST_PATH, time_frame, rebal_frequency)
net_exp_port.to_csv(FNAME_INDEX.format(BACKTEST_PATH, time_frame, rebal_frequency))